In [ ]:
%%pyspark
df = spark.read.load('abfss://test-env@steduanalyticscisd3b.dfs.core.windows.net/stage1/ed-fi/Assessment/data_d46f3789-8c9b-4b45-8a53-f1c266789fdf_bc1f569d-c4da-4ea4-ac06-5d833e34ecd0.parquet', format='parquet')
display(df.limit(10))

In [ ]:
storage_account = 'steduanalyticscisd3b'
use_test_env = True

In [ ]:
if use_test_env:
    stage1 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage1'
    stage2 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage2'
    stage3 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage3'
else:
    stage1 = 'abfss://stage1@' + storage_account + '.dfs.core.windows.net'
    stage2 = 'abfss://stage2@' + storage_account + '.dfs.core.windows.net'
    stage3 = 'abfss://stage3@' + storage_account + '.dfs.core.windows.net'

In [ ]:
# Create spark db to allow for access to the data in the delta-lake via SQL on-demand.
# This is only creating metadata for SQL on-demand, pointing to the data in the delta-lake.
# This also makes it possible to connect in Power BI via the azure sql data source connector.
def create_spark_db(db_name, source_path, tables):
    spark.sql('CREATE DATABASE IF NOT EXISTS ' + db_name)
    for table in tables:
        spark.sql("create table if not exists " + db_name + "." + table.table_name + " using PARQUET location '" + source_path + "/" + table.table_name + "'")

df = spark.read.format('parquet').load(stage2 + '/ed-fi/sys.tables')
tables = df.rdd.collect()
create_spark_db('s2_edfi', stage2 + '/ed-fi', tables)